# Gộp word với gloss

In [2]:
file_words = "/data/WordNet/WordNet_gốc/Words.txt"
file_gloss = "/data/WordNet/WordNet_gốc/gloss_tach.txt"
output_file = "D:\Semantic-Concept-Similarity\data\WordNet\WordNet.txt"

with open(file_words, "r", encoding="utf-8") as f1, open(file_gloss, "r", encoding="utf-8") as f2:
    words = f1.readlines()
    glosses = f2.readlines()

merged_lines = []
for w, g in zip(words, glosses):
    merged_lines.append(f"{w.strip()}\t{g.strip()}\n")

with open(output_file, "w", encoding="utf-8") as out:
    out.writelines(merged_lines)

print(output_file)


Đã gộp xong! File kết quả: D:\Semantic-Concept-Similarity\data\WordNet\WordNet.txt


<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:3: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:3: SyntaxWarning: invalid escape sequence '\S'
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9716\840028965.py:1: SyntaxWarning: invalid escape sequence '\S'
  file_words = "D:\Semantic-Concept-Similarity\data\WordNet\Words.txt"
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9716\840028965.py:2: SyntaxWarning: invalid escape sequence '\S'
  file_gloss = "D:\Semantic-Concept-Similarity\data\WordNet\gloss_tach.txt"
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9716\840028965.py:3: SyntaxWarning: invalid escape sequence '\S'
  output_file = "D:\Semantic-Concept-Similarity\data\WordNet\WordNet.txt"


# Truy vấn list word Vsim-400 vào kho tri thứ WordNet

### Tiền xử lý dử liệu đầu vào

In [1]:
import pandas as pd

In [2]:
file_path = "D:\Semantic-Concept-Similarity\data\BabelNet\Visim-400.txt"
df = pd.read_csv(file_path, sep="\t")
words = df[["Word1", "Word2"]].values.flatten()
output_path = "D:\Semantic-Concept-Similarity\data\WordNet\words_list.txt"
with open(output_path, "w", encoding="utf-8") as f:
    for w in words:
        f.write(str(w) + "\n")
print("Đã tạo file:", output_path)

Đã tạo file: D:\Semantic-Concept-Similarity\data\WordNet\words_list.txt


<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:4: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:4: SyntaxWarning: invalid escape sequence '\S'
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_4816\4239384428.py:1: SyntaxWarning: invalid escape sequence '\S'
  file_path = "D:\Semantic-Concept-Similarity\data\BabelNet\Visim-400.txt"
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_4816\4239384428.py:4: SyntaxWarning: invalid escape sequence '\S'
  output_path = "D:\Semantic-Concept-Similarity\data\WordNet\words_list.txt"


### Truy vấn Word list vào WordNet và lấy gloss của chúng


In [1]:
import os
Word_Net = r"D:\Semantic-Concept-Similarity\data\WordNet\WordNet.txt"
Word_list = r"D:\Semantic-Concept-Similarity\data\WordNet\words_list.txt"
output = r"D:\Semantic-Concept-Similarity\data\WordNet\WordNet_Definitions.txt"

with open(Word_list, "r", encoding="utf-8") as f:
    target_words = [line.strip() for line in f if line.strip()]

word_to_gloss = {}

with open(Word_Net, "r", encoding="utf-8") as f:
    for line in f:
        if not line.strip():
            continue
        parts = line.strip().split("\t", 1)
        if len(parts) == 2:
            word, gloss = parts
        else:
            # Nếu dòng không có gloss, gán gloss rỗng
            word, gloss = parts[0], "[]"
        word_to_gloss[word.lower()] = gloss

filtered_lines = []
for word in target_words:
    gloss = word_to_gloss.get(word.lower(), "[]")  # Nếu không có gloss thì []
    filtered_lines.append(f"{word}\t{gloss}\n")

with open(output, "w", encoding="utf-8") as f:
    f.writelines(filtered_lines)

print(f" Đã trích xuất {len(filtered_lines)} từ (theo đúng thứ tự). File lưu tại: {output}")


 Đã trích xuất 800 từ (theo đúng thứ tự). File lưu tại: D:\Semantic-Concept-Similarity\data\WordNet\WordNet_Definitions.txt


# Combine BabelNet WordNet

In [2]:
import re

file1 = r"D:\Semantic-Concept-Similarity\data\WordNet\WordNet_Definitions.txt"
file2 = r"D:\Semantic-Concept-Similarity\data\BabelNet\BabelNet_Defimitions.txt"
output = r"D:\Semantic-Concept-Similarity\data\ BabelNet_combine_WordNet\Word_Babel3.txt"

def read_babel_format(path):
    data = {}
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line or "," not in line:
                continue
            word, rest = line.split(",", 1)
            word = word.strip()
            glosses = re.findall(r"'(.*?)'", rest) # lấy tất cả trong ' '
            glosses = [g.strip() for g in glosses if g.strip()]
            data[word] = glosses
    return data

def read_wordnet_format(path):
    order = []
    data = {}
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if "\t" in line:
                word, gloss = line.split("\t", 1)
                word, gloss = word.strip(), gloss.strip()
                glosses = [g.strip() for g in re.split(r";|\.", gloss) if g.strip()]
                data[word] = glosses
                order.append(word)
            else:
                word = line.strip()
                data[word] = []
                order.append(word)
    return order, data

order, data1 = read_wordnet_format(file1)
data2 = read_babel_format(file2)

combined = {}
for word in order:
    g1 = data1.get(word, [])
    g2 = data2.get(word, [])

    if not g1 and not g2:
        combined[word] = []
    else:
        glosses = list(dict.fromkeys(g1 + g2))
        combined[word] = glosses

# kết quả
with open(output, "w", encoding="utf-8") as out:
    for word in order:
        glosses = combined.get(word, [])
        if glosses:
            gloss_str = "','".join(glosses)
            out.write(f"{word},['{gloss_str}']\n")
        else:
            out.write(f"{word},[]\n")

print(len(order))


800


# Tiền xử lý dữ liệu


### Xét thứ tự 2 dòng 1 , nếu 1 trong 2 không có gloss thì mình sẽ bỏ qua

In [1]:
input_file = r"D:\Semantic-Concept-Similarity\data\ BabelNet_combine_WordNet\Word_Babel3.txt"
output_file = r"D:\Semantic-Concept-Similarity\data\ BabelNet_combine_WordNet\WordNet_BabelNet_Definitions2.txt"

with open(input_file, "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f if line.strip()]

# kết quả lưu vào đây
result = []

# duyệt 2 dòng 1 lần
for i in range(0, len(lines) - 1, 2):
    line1 = lines[i]
    line2 = lines[i + 1]
    if "['[]']" in line1 or "['[]']" in line2 or ",['[]']" in line1 or ",['[]']" in line2:
        continue
    if "[]]" in line1 or "[]]" in line2 or line1.endswith("[]") or line2.endswith("[]"):
        continue
    result.extend([line1, line2])

# ghi ra file
with open(output_file, "w", encoding="utf-8") as f:
    for line in result:
        f.write(line + "\n")

print(f" Đã lọc xong còn: {len(result)}")


 Đã lọc xong còn: 602


# Tách WordNet_BabelNet_Definitions2.txt ra thành word và các gloss riêng

In [2]:
import  re

input = r"D:\Semantic-Concept-Similarity\data\ BabelNet_combine_WordNet\WordNet_BabelNet_Definitions2.txt"
output1= r"D:\Semantic-Concept-Similarity\data\ BabelNet_combine_WordNet\BCW_Word.txt"
output2= r"D:\Semantic-Concept-Similarity\data\ BabelNet_combine_WordNet\BCW_Definitions.txt"

with open(input, "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f if line.strip()]

words=[]
glosss=[]

for line in lines:
    try:
        #tach tu va gloss
        word, gloss = line.split(",", 1)
        word = word.strip()
        gloss = re.sub(r"[\[\]]","",gloss).strip()
        words.append(word)
        glosss.append(gloss)
    except ValueError:
     print(f"khong tach dc dong:{line}")

with open(output1, "w", encoding="utf-8") as fw:
    fw.writelines("\n".join(words))
with open(output2, "w", encoding="utf-8") as fg:
    fg.writelines("\n".join(glosss))


print(len(words))


602


In [3]:
print(len(glosss))

602
